### Coba
1. Upgrade OpenCV
2. Downgrade Python
3. Upgrade Python

In [1]:
#pip install --upgrade opencv-python

In [5]:
#Import Library
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

### Load Yaml

In [6]:
with open('Data.yaml',mode = 'r') as f:
    data_yaml = yaml.load(f,Loader = SafeLoader)

labels = data_yaml['names']
print(labels)

['container-number']


### Load YOLO Model

In [7]:
yolo = cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

#Keterangan Error
ERROR during processing node with 2 inputs and 3 outputs: 
        [Split]:(onnx_node!/model.24/Split) from domain='ai.onnx'

### Load Gambar Test Akhir

In [42]:
img = cv2.imread('./T (11).jpg')
image = img.copy()
row, col, d = image.shape

### Menerapkan YOLO Prediction Dari Gambar Test Akhir

In [43]:
#----1. Convert gambar ke square matriks
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row, 0:col] = image

#----2. Mendapatkan prediksi dari square array/square matriks
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB = True, crop = False)
yolo.setInput(blob)
preds = yolo.forward() #Deteksi/Prediksi dari YOLO

In [44]:
print(preds)

[[[8.23166275e+00 7.57426643e+00 1.53652237e+02 1.58997021e+01
   6.06972833e-07 9.99876618e-01]
  [1.70632267e+01 6.15354538e+00 1.03133904e+02 9.54354477e+00
   4.02473773e-07 9.99923468e-01]
  [2.63401623e+01 6.97766018e+00 7.41229401e+01 8.12462997e+00
   2.81796900e-07 9.99950886e-01]
  ...
  [5.56898621e+02 6.26341492e+02 1.72780670e+02 4.54231606e+01
   3.70157227e-06 9.99980211e-01]
  [5.94416443e+02 6.26758850e+02 1.43342041e+02 4.73014793e+01
   2.48572815e-06 9.99975085e-01]
  [6.16464478e+02 6.25793884e+02 1.07025696e+02 4.15984688e+01
   3.55326875e-07 9.99979734e-01]]]


In [45]:
print(preds.shape)

(1, 25200, 6)


### Non Maximum Supression

In [46]:
#----1. Filter deteksi berdasarkan nilai confidence (0.4) dan probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

#Width dan height dari gambar (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] #Confidences dari hasil deteksi objek
    #confidence = row[4] bekerja untuk .jpg tapi untuk .png tidak berhasil
    #Cek videonya, pake juga row[4] karena nilai confidence ada di colomn ke-5 (hitungannya mulai dari 0)
    #AttributeError: 'tuple' object has no attribute 'flatten' kalo nilai confidence = row[4], saat gambar .png
    if confidence > 0.4:
        class_score = row[5:].max() #Maximum probability dari 5 objek
        class_id = row[5:].argmax() #Mendapatkan posisi index saat max probability muncul
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            
            #Membuat Bounding Box dari value yang sudah didapatkan
            #Left, Top, Width, Height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left, top, width, height])
            
            #Append value dalam list kosong yang telah dibuat
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

#Cleaning duplikat value
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#NMS (Non Maximum Supression)
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45).flatten()

In [47]:
index

array([9])

In [48]:
confidences_np

[0.9355595707893372,
 0.7763668894767761,
 0.9304158687591553,
 0.7967761754989624,
 0.5075912475585938,
 0.8639713525772095,
 0.903584361076355,
 0.6601918935775757,
 0.9182114601135254,
 0.9388623237609863,
 0.9069558382034302,
 0.8988355994224548,
 0.9311953186988831]

In [49]:
boxes_np

[[875, 333, 499, 83],
 [875, 334, 500, 82],
 [876, 333, 496, 83],
 [875, 334, 500, 82],
 [875, 334, 492, 79],
 [877, 334, 495, 82],
 [880, 334, 494, 83],
 [876, 334, 493, 80],
 [877, 334, 498, 83],
 [880, 334, 493, 84],
 [877, 334, 497, 83],
 [876, 332, 497, 85],
 [877, 333, 498, 84]]

In [50]:
classes

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### Membuat Bounding Box Pada Gambar Test Akhir

In [51]:
for ind in index:
    #Ekstrak Bounding Box
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]+100) #Perlu evaluasi
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f'{class_name}: {bb_conf}%'
    print(text)

container-number: 100%


In [52]:
cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)
cv2.rectangle(image, (x,y-30), (x+w,y), (255,255,255), -1)
cv2.putText(image, text, (x,y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0), 1)

array([[[  8,   6,  18],
        [  4,   2,  14],
        [  5,   5,  19],
        ...,
        [ 70,  69,  48],
        [ 73,  76,  50],
        [ 77,  81,  52]],

       [[ 87,  89, 100],
        [ 89,  91, 102],
        [109, 110, 124],
        ...,
        [ 68,  67,  46],
        [ 70,  73,  47],
        [ 72,  76,  47]],

       [[178, 187, 197],
        [178, 188, 198],
        [182, 192, 202],
        ...,
        [ 67,  66,  46],
        [ 70,  73,  48],
        [ 70,  73,  47]],

       ...,

       [[ 17,  69,  99],
        [ 17,  69,  99],
        [ 17,  69,  99],
        ...,
        [120, 133, 109],
        [126, 139, 115],
        [131, 144, 120]],

       [[ 17,  69,  99],
        [ 17,  69,  99],
        [ 17,  69,  99],
        ...,
        [125, 137, 115],
        [133, 145, 123],
        [133, 145, 123]],

       [[ 17,  69,  99],
        [ 18,  70, 100],
        [ 18,  70, 100],
        ...,
        [129, 141, 119],
        [137, 149, 127],
        [132, 144, 122]]

In [53]:
cv2.imshow('ACNR',image)
cv2.imshow('Original',img)
cv2.waitKey(0)
cv2.destroyAllWindows

<function destroyAllWindows>

In [54]:
!pip list

Package                      Version
---------------------------- --------
absl-py                      1.4.0
anyio                        3.6.2
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
arrow                        1.2.3
astroid                      2.11.5
asttokens                    2.2.1
astunparse                   1.6.3
attrs                        23.1.0
autopep8                     2.0.1
backcall                     0.2.0
beautifulsoup4               4.12.2
bleach                       6.0.0
cachetools                   5.3.0
certifi                      2023.5.7
cffi                         1.15.1
charset-normalizer           3.1.0
colorama                     0.4.4
comm                         0.1.3
cycler                       0.11.0
debugpy                      1.6.7
decorator                    5.1.1
defusedxml                   0.7.1
dill                         0.3.5.1
distlib                      0.3.6
executing                    1.2.0
fas